In [ ]:
import yaml
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
import gensim
from gensim import corpora
from os import listdir
from os.path import isfile, join
from bs4 import BeautifulSoup
import requests
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import operator
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# LDA


<b> Atividades </b>

1) Faça download dos arquivos no link: https://drive.google.com/open?id=1PgGJ2OBFSKnr3O0RIgO2EvgbLh_avRJN

2) Carregue os quatro documentos. Para carregar um arquivo ``yml``, siga o exemplo abaixo:
```python
with open('data/chats/file.yml') as file:
    chat = yaml.load(file)
```
<b> Dica: adicione os 4 documentos em um único dicionário </b>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mypath = '/content/drive/MyDrive/Pratica/'

files = [ f for f in listdir(mypath) if isfile(join(mypath,f))]

In [ ]:
files 

['money.yml', 'proverbs.yml', 'linguistic_knowledge.yml', 'games.yml']

In [ ]:
all_chats ={}
for f in files:
  print("file",f)
  with open(mypath+f) as file:
    chat=yaml.load(file)
    all_chats[chat['categories'][0]]=chat['conversations']   

file money.yml
file proverbs.yml
file linguistic_knowledge.yml
file games.yml


<b> Atividade </b>

A variável ``chat`` é um dicionário, que possui a chave ``conversations``, que contém várias listas. Dentro de cada lista, existe um par de frases, uma escrita por um humano e outra escrita por um robô. Neste exercício, só iremos considerar a frase do humano, ou seja, a primeira posição do par nas listas.
Exemplo:

```python
chat = {'categories': ['compliment'],
        'conversations': [['você é linda', 'eu sei'],
                          ['adoro você', 'também'],
                          ['gosto da sua sinceridade', 'que bom']]}
```
No exemplo acima, iremos considerar apenas as frases: <b> 'você é linda', 'adoro você' e 'gosto da sua sinceridade' </b>.

Um documento no LDA é representado por uma única lista. Ou seja, as listas de frases selecionadas no exemplo anterior se tornarão uma única lista: <b> ['você é linda adoro você e gosto da sua sinceridade'] </b>



3) Conforme explicado anteriormente, faça as alterações necessárias no dado. Primeiro, considere apenas a frase da primeira posição, depois una todas as listas de frases em uma única lista.

In [ ]:
' '.join([chat[0] for chat in all_chats['games']])

"Você gosta de algum jogo? O que acha do Mario? O que acha do Sonic? O que acha do Pacman? O que acha do Assassin's Creed? O que acha do League of Legends? O que acha do Dota 2? Devo comprar um console? Mario ou Sonic? Conhece um programa chamado Discord? Conhece um programa chamado Team Speak? Conhece um programa chamado Skype? Você joga em consoles? Você acompanha o cenário de e-Sports? Você prefere PC ou console?"

In [ ]:
for k in all_chats.keys():
  all_chats[k] = ' '.join([chat[0] for chat in all_chats[k]])

In [ ]:
all_chats

{'conhecimento linguístico': 'o que é linguística? o que é um verbo? estuda quem foi Saussure? O que é uma Gramática O que é um substantivo? quem é Noam Chomsky? o que é um sujeito? Quem é Marcos Bagno? Quais os livros que todo estudante de Letras deve ter? Quais as áreas da Linguística? Quando uso o porque, o por que, o porquê e o porquê? Pode explicar o uso dos porquês? Qual a forma correta, concerteza ou com certeza? O que são as figuras de linguagem? o que é um pronome? o que é a ANPOLL? Quando será realizado o próximo SIC? Quais os pronomes pessoais do caso reto? Quem é o autor de Análise de Textos de Comunicação? O que é um advérbio? O que é a langue? O que é a parole O que é letramento? O que é a alfabetização?',
 'games': "Você gosta de algum jogo? O que acha do Mario? O que acha do Sonic? O que acha do Pacman? O que acha do Assassin's Creed? O que acha do League of Legends? O que acha do Dota 2? Devo comprar um console? Mario ou Sonic? Conhece um programa chamado Discord? Conh

<b> Atividade </b>

4) Faça uma limpeza nos dados carregados e processados do passo anterior. Aplique:
- tokenização
- lower
- remoção stopwords
- remoção pontuação

Ao final você deve obter uma lista com todos os documentos e em cada documento as palavras tokenizadas, exemplo:

```python
[['oi', 'tudo', 'bem'],
 ['mario', 'game', 'jogo']]
```

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def pre_processamento_texto(corpus):
  #print('Document')
  #print("#Tokenização")
  corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
  #lowcase
  corpus_alt = [ t.lower() for t in corpus_alt  ]
  #print('remove stopwords')
  portuguse_stops = stopwords.words('portuguese')
  corpus_alt = [t for  t in corpus_alt if t not in portuguse_stops]
  #print('remove numeros')
  corpus_alt = [re.sub(r'\d','',t) for t in  corpus_alt]
  #print('remove pontuação')
  corpus_alt = [t for  t in corpus_alt if t not in string.punctuation]
  #print('remove acentos')
  #corpus_alt = [unidecode(t) for t in  corpus_alt]
  return corpus_alt

In [ ]:
all_charts_limpo = []
for k in all_chats.keys():
  print(pre_processamento_texto(all_chats[k]))

['pago', 'mercado', 'ações', 'mercado', 'ações', 'mercado', 'ações', 'mercado', 'ações', 'mercado', 'ações', 'mercado', 'ações', 'mercado', 'ações', 'mercado', 'ações', 'taxa', 'juros', 'dólar', 'dinheiro', 'mercado', 'ações', 'mercado', 'ações', 'mercado', 'ações', 'investimento', 'favorito', 'economia', 'economia', 'economia', 'faço', 'estoque', 'dinheiro', 'quanto', 'ganha', 'quanto', 'ganha', 'quanto', 'ganha', 'quanto', 'cobra', 'quanto', 'dinheiro', 'quanto', 'dinheiro', 'quanto', 'dinheiro', 'dólar', 'real', 'dono', 'público']
['casa', 'médico', 'todos', 'doentes', 'insistiu', 'tanto', 'conseguiu', 'casar', 'precisei', 'pregar', 'prego', 'martelo', 'então', 'preguei', 'pequena', 'barra', 'ferro', 'ganhei', 'livro', 'quinze', 'queria', 'sagarana', 'sozinha', 'conseguirei', 'concluir', 'trabalho', 'tudo', 'combinado', 'festa', 'pai', 'chegou', 'ser', 'paciente', 'conversar', 'cedo', 'tarde', 'verdade', 'vai', 'aparecer', 'machocou', 'agora', 'est', '·', 'sendo', 'machucado', 'melh

In [ ]:
all_charts_limpo = []
for k in all_chats.keys():
 all_charts_limpo.append(pre_processamento_texto(all_chats[k]))

In [ ]:
all_charts_limpo

[['pago',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'taxa',
  'juros',
  'dólar',
  'dinheiro',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'investimento',
  'favorito',
  'economia',
  'economia',
  'economia',
  'faço',
  'estoque',
  'dinheiro',
  'quanto',
  'ganha',
  'quanto',
  'ganha',
  'quanto',
  'ganha',
  'quanto',
  'cobra',
  'quanto',
  'dinheiro',
  'quanto',
  'dinheiro',
  'quanto',
  'dinheiro',
  'dólar',
  'real',
  'dono',
  'público'],
 ['casa',
  'médico',
  'todos',
  'doentes',
  'insistiu',
  'tanto',
  'conseguiu',
  'casar',
  'precisei',
  'pregar',
  'prego',
  'martelo',
  'então',
  'preguei',
  'pequena',
  'barra',
  'ferro',
  'ganhei',
  'livro',
  'quinze',
  'queria',
  'sagarana',
  'sozinha',
  'conseguirei',
  'concluir',
  'trabalho',
  'tudo',
  'combinado',
  'f

<b> Atividade </b>

5) Crie um dicionário com nosso corpus, de forma que cada termo único (valor) possua um índice (chave). 

Dica: 
- utilize o ```gensim.corpora.Dictionary ``` para criar este dicionário. 
- utilize o texto pré-processado

In [ ]:
dicionario = gensim.corpora.Dictionary(all_charts_limpo)

In [ ]:
len(dicionario)

244

In [ ]:
dicionario.doc2bow(all_charts_limpo[2])

[(146, 1),
 (167, 1),
 (168, 1),
 (169, 1),
 (170, 1),
 (171, 1),
 (172, 1),
 (173, 1),
 (174, 1),
 (175, 1),
 (176, 1),
 (177, 1),
 (178, 1),
 (179, 1),
 (180, 1),
 (181, 1),
 (182, 1),
 (183, 1),
 (184, 1),
 (185, 1),
 (186, 1),
 (187, 1),
 (188, 1),
 (189, 1),
 (190, 2),
 (191, 1),
 (192, 1),
 (193, 1),
 (194, 1),
 (195, 1),
 (196, 1),
 (197, 1),
 (198, 2),
 (199, 1),
 (200, 1),
 (201, 1),
 (202, 1),
 (203, 3),
 (204, 1),
 (205, 1),
 (206, 1),
 (207, 1),
 (208, 1),
 (209, 1),
 (210, 1),
 (211, 1),
 (212, 2),
 (213, 1),
 (214, 1)]

<b> Atividade </b>

6) Converta os documentos em índices, utilizando o método ```doc2bow``` em cada documento.

In [ ]:
chat_bow = [dicionario.doc2bow(chat) for chat in all_charts_limpo]

<b> Atividade </b>

7) Aplique o modelo LDA do Gensim nos dados processados acima. O modelo recebe a matriz de índices gerada anteriormente.
Abaixo, apresentamos um exemplo de como aplicar o modelo.


```python
model = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=4, id2word = dictionary, passes=1000, random_state=123)
#para mostrar os tópicos:
model.show_topics()
```

Fique a vontade para alterar os parâmetros e fazer testes.

In [ ]:
model = gensim.models.ldamodel.LdaModel(chat_bow , num_topics=4, id2word = dicionario, passes=1000, random_state=123)
# para mostrar os t

In [ ]:
model.show_topics()

[(0,
  '0.028*"quais" + 0.020*"uso" + 0.020*"porquê" + 0.020*"linguística" + 0.011*"parole" + 0.011*"noam" + 0.011*"pode" + 0.011*"pessoais" + 0.011*"verbo" + 0.011*"marcos"'),
 (1,
  '0.015*"ter" + 0.015*"vou" + 0.010*"comprar" + 0.010*"·" + 0.010*"bem" + 0.010*"faça" + 0.010*"tudo" + 0.010*"quer" + 0.010*"pai" + 0.010*"feito"'),
 (2,
  '0.099*"mercado" + 0.099*"ações" + 0.064*"quanto" + 0.046*"dinheiro" + 0.029*"economia" + 0.029*"ganha" + 0.020*"dólar" + 0.011*"real" + 0.011*"público" + 0.011*"pago"'),
 (3,
  '0.060*"acha" + 0.031*"conhece" + 0.031*"programa" + 0.031*"chamado" + 0.021*"sonic" + 0.021*"mario" + 0.021*"console" + 0.012*"skype" + 0.012*"prefere" + 0.012*"pc"')]

<b> Atividade </b>

8) Para responder esta questão utilize os parâmetros sugeridos na questão 7. Analise os grupos gerados anteriormente e responda:
- a) Os grupos são bem dividos? Caracterize-os.
- b) Dê um nome para cada grupo de tópicos.

A) Foi considerado 4 tópicos, um tópico por texto. Os grupo foram bem dividos em tópicos conforme os textos repassados.

B) Grupo 0 = mercado financeiro
Grupo 1 = linguistica
Grupo 2 = Proverbios
grupo 3 = Games

# Sumarização

Aqui iremos realizar uma sumarização de notíticas com técnicas que vimos nas aulas anteriores.

<b> Atividade </b>

9) Faça download do texto da seguinte notícia: 

``https://g1.globo.com/tecnologia/noticia/2022/03/12/instagram-restrito-na-russia-entenda-a-importancia-da-rede-social-para-o-pais-de-putin.ghtml``

Dica: para fazer download, consulte a Aula Prática 4.

In [ ]:
def match_class(target):                                                        
    def do_match(tag):                                                          
        classes = tag.get('class', [])                                          
        return all(c in classes for c in target)                                
    return do_match 

def get_text_url(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    #remove marcações de scripts e style
    texto = soup.find_all(match_class(["content-text__container"]))
    all_text = ""
    for t in texto:
        all_text += t.get_text()
    return all_text

In [ ]:
texto_noticia = get_text_url('https://g1.globo.com/tecnologia/noticia/2022/03/12/instagram-restrito-na-russia-entenda-a-importancia-da-rede-social-para-o-pais-de-putin.ghtml')

<b> Atividade </b>

10) Imprima o texto coletado.

In [ ]:
print(texto_noticia)

 A decisão de bloquear o Instagram na Rússia deve ter impactos que vão além da disputa entre o governo russo e a companhia americana, já que o aplicativo é bastante popular no país. A rede social tem 80 milhões de usuários russos, segundo o diretor Adam Mosseri.  A Rússia anunciou que vai restringir o acesso ao Instagram na próxima segunda-feira (14) como resposta à mudança de política para discurso de ódio da Meta, controladora do Instagram e também das plataformas Facebook e WhatsApp.  Na última sexta-feira (11), a empresa americana mudou temporariamente suas regras para permitir que usuários das redes sociais em alguns países defendam atos de violência contra russos no contexto da guerra na Ucrânia (saiba mais ao fim da reportagem).  De acordo com o comunicado do governo russo, a decisão de bloquear o app na segunda dá aos usuários 48 horas para transferirem suas fotos e vídeos para outras redes sociais e notificarem seus contatos e clientes.  Os russos usam muito mais o Instagram d

<b> Atividade </b>

11) Faça a tokenização das sentenças.

In [ ]:
 nltk
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
texto_sents = sent_tokenize(texto_noticia)

In [ ]:
texto_sents

[' A decisão de bloquear o Instagram na Rússia deve ter impactos que vão além da disputa entre o governo russo e a companhia americana, já que o aplicativo é bastante popular no país.',
 'A rede social tem 80 milhões de usuários russos, segundo o diretor Adam Mosseri.',
 'A Rússia anunciou que vai restringir o acesso ao Instagram na próxima segunda-feira (14) como resposta à mudança de política para discurso de ódio da Meta, controladora do Instagram e também das plataformas Facebook e WhatsApp.',
 'Na última sexta-feira (11), a empresa americana mudou temporariamente suas regras para permitir que usuários das redes sociais em alguns países defendam atos de violência contra russos no contexto da guerra na Ucrânia (saiba mais ao fim da reportagem).',
 'De acordo com o comunicado do governo russo, a decisão de bloquear o app na segunda dá aos usuários 48 horas para transferirem suas fotos e vídeos para outras redes sociais e notificarem seus contatos e clientes.',
 'Os russos usam muito 

<b> Atividade </b>

12) Aplique o pré-processamento nos dados:

- tokenização
- lower
- remoção stopwords
- remoção pontuação

In [ ]:
def pre_processamento_texto(corpus):
  #print('Document')
  #print("#Tokenização")
  corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
  #lowcase
  corpus_alt = [ t.lower() for t in corpus_alt  ]
  #print('remove stopwords')
  portuguse_stops = stopwords.words('portuguese')
  corpus_alt = [t for  t in corpus_alt if t not in portuguse_stops]
  #print('remove numeros')
  corpus_alt = [re.sub(r'\d','',t) for t in  corpus_alt]
  #print('remove pontuação')
  corpus_alt = [t for  t in corpus_alt if t not in string.punctuation]
  #print('remove acentos')
  #corpus_alt = [unidecode(t) for t in  corpus_alt]
  return corpus_alt

In [ ]:
texto_limpo = [pre_processamento_texto(sent) for sent in texto_sents]

<b> Atividade </b>

13) Agora iremos implementar a sumarização. Para sumarizar você precisará implementar um método para cada uma das orientações abaixo:

13.1) Faça um método que calcule a similaridade entre duas sentenças utilizando a similaridade do conseno. 

Dica: faça a tranformação aplicando Bag of Words. Consulte a Aula 3.

In [ ]:
f0 = texto_limpo[0]
f1 = texto_limpo[1]

In [ ]:
todas_palavras = f1+f0

In [ ]:
vect_bag = CountVectorizer(binary = True, analyzer='word')

In [ ]:
vect_bag.fit(todas_palavras)

CountVectorizer(binary=True)

In [ ]:
f0_bow = vect_bag.transform([' '.join(f0)])
f1_bow = vect_bag.transform([' '.join(f1)])

In [ ]:
cosine_similarity(f0_bow,f1_bow)

array([[0.]])

In [ ]:
def calcula_similaridade(f1,f0):
    todas_palavras = f1+f0
    vect_bag = CountVectorizer(binary = True, analyzer='word')
    vect_bag.fit(todas_palavras)
    f0_bow = vect_bag.transform([' '.join(f0)]).todense()
    f1_bow = vect_bag.transform([' '.join(f1)]).todense()    
    return cosine_similarity(f0_bow,f1_bow)[0][0]

In [ ]:
calcula_similaridade(texto_limpo[0],texto_limpo[1])

0.0

13.2) Faça um método para construir uma matriz de similaridades entre as sentenças. 

Exemplo, seu texto tem 10 sentenças, você deve construir uma matriz (10,10). A posição (0,1) da matriz corresponderá a similaridade do cosseno entre a sentença 0 e a sentença 1.

In [ ]:
matriz_similaridade = np.zeros((len(texto_sents),len(texto_sents)))

In [ ]:
for linha in range(0,len(texto_sents)):
  for coluna in range(0,len(texto_sents)):
    if linha==coluna:
      continue
    matriz_similaridade[linha][coluna]  =  calcula_similaridade(texto_limpo[linha],texto_limpo[coluna])

In [ ]:
 import pandas as pd
 pd.DataFrame(matriz_similaridade)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.000000,0.000000,0.108148,0.047140,0.210819,0.058926,0.166667,0.223607,0.251976,0.0,...,0.136083,0.089087,0.149071,0.136083,0.129099,0.121716,0.000000,0.000000,0.000000,0.105409
1,0.000000,0.000000,0.000000,0.133333,0.074536,0.416667,0.117851,0.316228,0.089087,0.0,...,0.000000,0.000000,0.000000,0.000000,0.060858,0.258199,0.000000,0.000000,0.000000,0.074536
2,0.108148,0.000000,0.000000,0.045883,0.051299,0.114708,0.162221,0.145095,0.122628,0.0,...,0.198680,0.173422,0.145095,0.662266,0.041885,0.059235,0.000000,0.122628,0.051299,0.102598
3,0.047140,0.133333,0.045883,0.000000,0.134164,0.100000,0.070711,0.126491,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.182574,0.154919,0.000000,0.160357,0.044721,0.134164
4,0.210819,0.074536,0.051299,0.134164,0.000000,0.055902,0.000000,0.070711,0.059761,0.0,...,0.000000,0.000000,0.000000,0.064550,0.000000,0.057735,0.000000,0.059761,0.000000,0.050000
5,0.058926,0.416667,0.114708,0.100000,0.055902,0.000000,0.176777,0.237171,0.133631,0.0,...,0.144338,0.000000,0.079057,0.144338,0.091287,0.258199,0.000000,0.000000,0.000000,0.055902
6,0.166667,0.117851,0.162221,0.070711,0.000000,0.176777,0.000000,0.335410,0.283473,0.0,...,0.306186,0.000000,0.111803,0.204124,0.000000,0.182574,0.000000,0.000000,0.079057,0.079057
7,0.223607,0.316228,0.145095,0.126491,0.070711,0.237171,0.335410,0.000000,0.338062,0.0,...,0.182574,0.119523,0.200000,0.091287,0.057735,0.081650,0.000000,0.084515,0.000000,0.000000
8,0.251976,0.089087,0.122628,0.000000,0.059761,0.133631,0.283473,0.338062,0.000000,0.0,...,0.154303,0.101015,0.169031,0.154303,0.048795,0.069007,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


13.3) - Depois que construímos a matriz de similaridades, faça o ranking das sentenças utilizando o método ``pagerank`` do pacote `networkx`. Este método irá retornar um score para cada sentença. Para utilizar este método, os dados devem representados em um grafo. Para isto converta a matriz de similaridades para um grafo de similaridades, utilizando o método `network.from_numpy_array(matrix)`.

<i>Mais informações Pagerank: </i>https://www.agenciamestre.com/marketing-digital/o-que-e-pagerank/

In [ ]:
grafo = nx.from_numpy_array(matriz_similaridade)

In [ ]:
nx.pagerank(grafo)

{0: 0.058557206426263074,
 1: 0.04779433767142721,
 2: 0.06828702806080768,
 3: 0.04085724151197877,
 4: 0.028934579035842803,
 5: 0.06502392512958774,
 6: 0.06197851263046078,
 7: 0.07498935386389034,
 8: 0.05586699428765852,
 9: 0.006479481641468792,
 10: 0.020838203553527307,
 11: 0.02275715046841103,
 12: 0.03422983891144631,
 13: 0.024795375729986607,
 14: 0.043182775234267376,
 15: 0.03784204178180282,
 16: 0.045236605672114176,
 17: 0.06793146143310948,
 18: 0.03537505215387676,
 19: 0.053728822604191234,
 20: 0.009994482261736024,
 21: 0.033346801534199756,
 22: 0.018668256086341022,
 23: 0.04330447231560399}

In [ ]:
sents_scores = nx.pagerank(grafo)

13.4) Ordene os scores retornados pelo `pagerank` e imprima as <b> 5 </b> sentenças com maior score.

In [ ]:
scores_sorted = dict(sorted(sents_scores.items(),key = operator.itemgetter(1),reverse = True))

In [ ]:
' '.join([texto_sents[id_sent] for id_sent in sorted(list(scores_sorted)[:5])])

'A Rússia anunciou que vai restringir o acesso ao Instagram na próxima segunda-feira (14) como resposta à mudança de política para discurso de ódio da Meta, controladora do Instagram e também das plataformas Facebook e WhatsApp. Os russos usam muito mais o Instagram do que o Facebook, a rede social com mais usuários no mundo, segundo dados da consultoria eMarketer publicados pela agência France Presse. O Facebook tinha 7,5 milhões naquele país em 2021, o equivalente a 7,3% dos internautas, contra 51 milhões no Instagram. O Instagram não divulga dados de usuários por país, mas, na sexta, Mosseri postou que são 80 milhões na Rússia. A decisão é uma resposta à mudança de política sobre discurso de ódio da Meta, controladora do Facebook, do WhatsApp e do Instagram.'

<b> Atividade </b>

14) Analise o resultado da sumarização, construindo uma crítica sobre a qualidade e relevância da mesma.

O sumário construído leva ao entendimento geral sobre o conteudo da noticia, com frases mais relevantes, falando sobre a restrição do acesso ao instagram na rússia e a quantidade de usuários, demonstrando a popularidade do aplicativo na russia de forma resumida. 

<b> Atividade </b>

15) Faça o exercício 13 mais uma vez, agora substituindo a representação BoW pela TFIDF, usando o TDIDFVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def calcula_similaridade_tfidf(f1,f0):
    todas_palavras = f1+f0
    vect = TfidfVectorizer()
    vect.fit(todas_palavras)
    f0_tfidf = vect.transform([' '.join(f0)]).todense()
    f1_tfidf = vect.transform([' '.join(f1)]).todense()    
    return cosine_similarity(f0_tfidf,f1_tfidf)[0][0]

In [ ]:
calcula_similaridade_tfidf(texto_limpo[0],texto_limpo[1])

0.0

In [ ]:
matriz_similaridade_tfidf = np.zeros((len(texto_sents),len(texto_sents)))

In [ ]:
for linha in range(0,len(texto_sents)):
  for coluna in range(0,len(texto_sents)):
    if linha==coluna:
      continue
    matriz_similaridade_tfidf[linha][coluna]  =  calcula_similaridade_tfidf(texto_limpo[linha],texto_limpo[coluna])

In [ ]:
 import pandas as pd
 pd.DataFrame(matriz_similaridade_tfidf)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.000000,0.000000,0.114459,0.038607,0.177210,0.047764,0.120698,0.186049,0.233145,0.0,...,0.111355,0.071500,0.121946,0.111355,0.125558,0.087692,0.000000,0.000000,0.000000,0.086979
1,0.000000,0.000000,0.000000,0.110130,0.060129,0.361029,0.137000,0.262507,0.066243,0.0,...,0.000000,0.000000,0.000000,0.000000,0.046581,0.281383,0.000000,0.000000,0.000000,0.060365
2,0.114459,0.000000,0.000000,0.035581,0.039600,0.121131,0.150810,0.152609,0.152622,0.0,...,0.194141,0.135195,0.152609,0.625124,0.052513,0.068276,0.000000,0.095234,0.037657,0.080126
3,0.038607,0.110130,0.035581,0.000000,0.111965,0.082568,0.050853,0.104394,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.144342,0.113116,0.000000,0.133777,0.034867,0.112185
4,0.177210,0.060129,0.039600,0.111965,0.000000,0.045431,0.000000,0.057092,0.045063,0.0,...,0.000000,0.000000,0.000000,0.052223,0.000000,0.041233,0.000000,0.048458,0.000000,0.040917
5,0.047764,0.361029,0.121131,0.082568,0.045431,0.000000,0.127740,0.197006,0.101567,0.0,...,0.117823,0.000000,0.063357,0.117823,0.070638,0.261429,0.000000,0.000000,0.000000,0.045564
6,0.120698,0.137000,0.150810,0.050853,0.000000,0.127740,0.000000,0.306349,0.241165,0.0,...,0.225145,0.000000,0.078455,0.146894,0.000000,0.116256,0.000000,0.000000,0.053848,0.056665
7,0.186049,0.262507,0.152609,0.104394,0.057092,0.197006,0.306349,0.000000,0.314919,0.0,...,0.148119,0.093838,0.161894,0.072523,0.075849,0.057733,0.000000,0.067448,0.000000,0.000000
8,0.233145,0.066243,0.152622,0.000000,0.045063,0.101567,0.241165,0.314919,0.000000,0.0,...,0.116982,0.125556,0.172945,0.116982,0.106884,0.045571,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
grafo_tfidf = nx.from_numpy_array(matriz_similaridade_tfidf)

In [ ]:
nx.pagerank(grafo_tfidf)

{0: 0.05758376107004353,
 1: 0.049144204298186694,
 2: 0.07477088421526565,
 3: 0.038789261304395695,
 4: 0.02774903184432527,
 5: 0.0643024987876595,
 6: 0.05772978693584098,
 7: 0.07506149604625553,
 8: 0.06082456559630635,
 9: 0.006479481641468792,
 10: 0.020402398932465392,
 11: 0.02153571536602076,
 12: 0.0344546991034485,
 13: 0.024538109034475085,
 14: 0.04214085645944586,
 15: 0.03805772834019138,
 16: 0.04504334301079841,
 17: 0.0658873403749851,
 18: 0.041528081788428865,
 19: 0.05202899731811837,
 20: 0.009675331511407391,
 21: 0.03275682216353686,
 22: 0.017753154209681214,
 23: 0.0417624506472486}

In [ ]:
sents_scores_tfidf = nx.pagerank(grafo)

In [ ]:
scores_sorted_tfidf  = dict(sorted(sents_scores_tfidf.items(),key = operator.itemgetter(1),reverse = True))

In [ ]:
' '.join([texto_sents[id_sent] for id_sent in sorted(list(scores_sorted_tfidf )[:5])])

'A Rússia anunciou que vai restringir o acesso ao Instagram na próxima segunda-feira (14) como resposta à mudança de política para discurso de ódio da Meta, controladora do Instagram e também das plataformas Facebook e WhatsApp. Os russos usam muito mais o Instagram do que o Facebook, a rede social com mais usuários no mundo, segundo dados da consultoria eMarketer publicados pela agência France Presse. O Instagram não divulga dados de usuários por país, mas, na sexta, Mosseri postou que são 80 milhões na Rússia. "Essa decisão cortará 80 milhões na Rússia uns dos outros e do resto do mundo, já que cerca de 80% das pessoas na Rússia seguem uma conta do Instagram fora de seu país. A decisão é uma resposta à mudança de política sobre discurso de ódio da Meta, controladora do Facebook, do WhatsApp e do Instagram.'

<b> Atividade </b>

16) Faça uma comparação dos resultados de sumarização obtidos pelas técnicas de Bag of Words e TFIDF, explicitando as diferenças observadas de forma concreta e exemplificando suas análises com frases dos textos.

In [ ]:
' '.join([texto_sents[id_sent] for id_sent in sorted(list(scores_sorted)[:5])])

'A Rússia anunciou que vai restringir o acesso ao Instagram na próxima segunda-feira (14) como resposta à mudança de política para discurso de ódio da Meta, controladora do Instagram e também das plataformas Facebook e WhatsApp. Os russos usam muito mais o Instagram do que o Facebook, a rede social com mais usuários no mundo, segundo dados da consultoria eMarketer publicados pela agência France Presse. O Instagram não divulga dados de usuários por país, mas, na sexta, Mosseri postou que são 80 milhões na Rússia. "Essa decisão cortará 80 milhões na Rússia uns dos outros e do resto do mundo, já que cerca de 80% das pessoas na Rússia seguem uma conta do Instagram fora de seu país. A decisão é uma resposta à mudança de política sobre discurso de ódio da Meta, controladora do Facebook, do WhatsApp e do Instagram.'

In [ ]:
' '.join([texto_sents[id_sent] for id_sent in sorted(list(scores_sorted_tfidf )[:5])])

'A Rússia anunciou que vai restringir o acesso ao Instagram na próxima segunda-feira (14) como resposta à mudança de política para discurso de ódio da Meta, controladora do Instagram e também das plataformas Facebook e WhatsApp. Os russos usam muito mais o Instagram do que o Facebook, a rede social com mais usuários no mundo, segundo dados da consultoria eMarketer publicados pela agência France Presse. O Instagram não divulga dados de usuários por país, mas, na sexta, Mosseri postou que são 80 milhões na Rússia. "Essa decisão cortará 80 milhões na Rússia uns dos outros e do resto do mundo, já que cerca de 80% das pessoas na Rússia seguem uma conta do Instagram fora de seu país. A decisão é uma resposta à mudança de política sobre discurso de ódio da Meta, controladora do Facebook, do WhatsApp e do Instagram.'

A utilização das duas técnicas resultou em sumários semelhantes.

<b> Desafio! (opcional, valendo pontos extras) </b>

Implemente e analise os resultados de uma solução que faça a sumarização dos textos utilizando Word Embeddings + WMD